### 基礎設定

In [ ]:
import os
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4o-mini")

### 模型動態選擇

In [ ]:
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=2)
tools = [web_search]

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain_core.messages import HumanMessage

# --- ① 準備兩種 OpenAI 模型 (一個省錢，一個聰明) ---

basic_model = ChatOpenAI(model="gpt-4o-mini")
reasoner_model = ChatOpenAI(model="gpt-4o")


def _get_last_user_text(messages) -> str:
    """
    [輔助工具]
    從一堆對話紀錄(messages)中，挖出「使用者講的最後一句話」。
    (如果找不到，就回傳一個空字串 "" )
    """
    # 從後面往前翻 (reversed)
    for m in reversed(messages):
        # 抓到 HumanMessage (使用者)
        if isinstance(m, HumanMessage):
            # content 可能是純文字，也可能是 [圖, 文] 這種複雜格式
            # 我們這裡只處理最常見的「純文字」情況
            return m.content if isinstance(m.content, str) else ""
    return ""


# 分流器
@wrap_model_call
def dynamic_openai_routing(request: ModelRequest, handler) -> ModelResponse:
    """
    (這是一個 LangChain 中間件 Middleware)
    
    💡 核心邏輯：動態 AI 模型路由！
    
    根據對話的「複雜度」，自動切換要 call 哪一個 OpenAI 模型。
    - 複雜問題 (例如數學、寫程式) -> gpt-4o
    - 簡單問題 (例如哈囉、聊天) -> gpt-4o-mini
    """
    
    # 先從 request 裡面把完整的對話紀錄(messages)挖出來
    messages = request.state.get("messages", [])
    msg_count = len(messages) # 總共講了幾句話
    
    # 抓出使用者最後一句話
    last_user = _get_last_user_text(messages)
    last_len = len(last_user) # 最後一句話有多長

    # ----------------------------------------------------
    # 🕵️‍♂️ 判斷「複雜任務」的關鍵詞 (你可以自己多加一點)
    # ----------------------------------------------------
    hard_keywords = (
        "證明", "推導", "嚴謹", "規劃", "多步驟", "演算法", "優化",
        "chain of thought", "step-by-step", "reason step by step",
        "數學", "邏輯證明", "約束求解", "寫程式", "debug"
    )

    # ----------------------------------------------------
    # 💡 判斷是否為「困難模式」的 Heuristics
    # ----------------------------------------------------
    is_hard = (
        msg_count > 10 or  # 1. 已經聊很久了 (可能進入深度討論)
        last_len > 120 or  # 2. 使用者一口氣問了超長的問題
        any(kw.lower() in last_user.lower() for kw in hard_keywords) # 3. 困難關鍵詞
    )

    # ----------------------------------------------------
    # 🚦 決定要派誰出場！
    # ----------------------------------------------------
    if is_hard:
        # 判定為困難
        request.model = reasoner_model
    else:
        # 只是小問題
        request.model = basic_model

    # 🚀 把 request (已經被我們偷換掉 model) 丟給下一棒 (handler)
    # handler 就是「真正去 call AI」的人
    return handler(request)

# ② 創建 Agent
agent = create_agent(
    model=basic_model,
    tools=tools,  
    middleware=[dynamic_openai_routing]
)

In [ ]:
from langchain_core.messages import HumanMessage

simple_question = "請問台北市長是誰？"
simple_response = agent.invoke({"messages": [HumanMessage(content=simple_question)]})

simple_response

In [ ]:
complex_question = """
請幫我詳細推理以下數學問題：
假設有一個粒子以恆定加速度 $$ a $$ 沿著直線運動，初速度為 $$ v_0 $$，位移為 $$ s $$。
請推導出其速度和時間 $$ t $$ 之間的函數關係，並且逐步解釋每一個步驟的物理意義義。
"""
complex_response = agent.invoke({"messages": [HumanMessage(content=complex_question)]})

complex_response

### 裁剪訊息

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langchain.messages import RemoveMessage, SystemMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime
from typing import Any
from langchain.tools import tool

# --- 1. 初始化 AI 模型 ---
model = ChatOpenAI(model="gpt-4o-mini")


# --- 2. 關鍵的「記憶體裁剪」 (Middleware) ---
@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """
    在把對話紀錄(state)丟給 AI 動腦之前，
    我們先在這裡動手腳，把紀錄「砍到」只剩精華。
    
    為什麼要這麼做？
    1. 省錢！ (Token 越少越省錢)
    2. 避免 AI 記憶體爆炸 (Context Window)
    """
    
    # 先把到目前為止的「完整對話紀錄」(一長串) 拿出來
    messages = state["messages"]

    # ----------------------------------------------------
    # 我們的「裁剪規則」：
    # 保留「第一則 (人設)」 + 「最後三則 (最新對話)」
    # ----------------------------------------------------
    KEEP_FIRST = 1 
    KEEP_LAST = 3
    TOTAL_KEPT = KEEP_FIRST + KEEP_LAST # 總共要留 4 則

    # 如果對話還很短 (小於等於 4 句)
    if len(messages) <= TOTAL_KEPT:
        return None  # 回傳 None 代表「沒事，照常執行」

    # --- 開始裁剪！ ---
    
    # 保留第 0 則訊息 (這 99% 是我們的「系統提示」或「人設」)
    first_msg = messages[0] 
    
    # 抓出最後 3 則訊息 (最新的對話)
    last_three_msgs = messages[-KEEP_LAST:]

    # 組合出我們的重要的對話紀錄
    new_messages = [first_msg] + last_three_msgs

    print(
        f"\n✂️ 原 {len(messages)} 則 → 砍到剩 {len(new_messages)} 則\n"
    )

    # 回傳一個字典，告訴 Agent 執行器 (Runtime) 該怎麼更新 state
    return {
        "messages": [
            # 1. 先把舊的紀錄「全部洗掉」(一鍵清空)！
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            # 2. 再把我們整理好的訊息塞回去
            *new_messages
        ]
    }

# --- 4. 建立 Agent ---
agent = create_agent(
    model=model,              
    tools=tools,                 
    middleware=[trim_messages],  
    checkpointer=InMemorySaver(),
)

In [ ]:
config = {
    "configurable": {
        "thread_id": "2"  
    }
}

# 第一次對話
agent.invoke(
    {"messages": "你是小豬豬"},
    config
)

In [ ]:
agent.invoke(
    {"messages": "給我一句心靈雞湯"},
    config
)

In [ ]:
agent.invoke(
    {"messages": "你是誰？"},
    config
)

### 練習

- Summarization
- Human In The Loop